# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='../Reacher.app')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Training the Agent

The agent is now trained to solve the environment.

In [ ]:
from collections import deque
from Agent import Agent
from unityagents import UnityEnvironment
import numpy as np
import torch
import pickle

MONITOR_INTERVAL = 10
TRAIN_MODE = True

env_info = env.reset(train_mode=TRAIN_MODE)[brain_name]
state = env_info.vector_observations[0]
done = False
agent = Agent(state_size, action_size, seed=0)

scores_window = deque(maxlen=100)
success_score = 5
scores = []
i_episode = 1
print(" {0} {0}\n|   Episode Number\t|   Avg Score({1})\t|\n {2} {2}".format('_'*23,MONITOR_INTERVAL,'-'*23))

while success_score <= 30:
    env_info = env.reset(train_mode=TRAIN_MODE)[brain_name]
    state = env_info.vector_observations[0]
    score = 0
    done = False
    while not done:
        action = agent.get_action(state,add_noise=TRAIN_MODE)[0]
        env_info = env.step(action)[brain_name]
        next_state = env_info.vector_observations[0]
        reward = env_info.rewards[0]
        done = env_info.local_done[0]
        agent.step(state,action,reward,next_state,done)
        score += reward
        state = next_state
    scores_window.append(score)
    scores.append(score)

    #Monitor the training over an interval.
    if i_episode%MONITOR_INTERVAL == 0:
        print('\r|   {0}\t{1}|   {2:.2f}\t\t|\n {3} {3}'.format(i_episode, '\t\t' if i_episode < 1000 else '\t',  np.mean(list(scores_window[i] for i in range(-1*MONITOR_INTERVAL,0))),'-'*23))
    
    #Display the averge score ever 100 episodes.
    if i_episode%100 == 0:
        print('\t\tAverage Score: {:.2f}'.format(np.mean(scores_window)))
        print(" {0} {0}\n|   Episode Number\t|   Avg Score({1})\t|\n {2} {2}".format('_'*23,MONITOR_INTERVAL,'-'*23))

    #Create periodic checkpoints incase of error.
    if i_episode%1000 == 0:
        torch.save(agent.actor_local.state_dict(), 'checkpoint/actor.pth'.format(i_episode-100))
        torch.save(agent.critic_local.state_dict(), 'checkpoint/critic.pth'.format(i_episode-100))
        success_score+=1
        with open('checkpoint/scores.pkl'.format(i_episode-100),'wb') as f:
            pickle.dump(scores,f)
    
    #Calculate the current average score and check that of it is greater than the current target.
    if np.mean(scores_window) >= success_score:
        print('{}\n Environment Solved in {:d} episodes\n Average Score: {:.2f}\n{}'.format('*'*48,i_episode-100,np.mean(scores_window),'*'*48))
        torch.save(agent.actor_local.state_dict(), 'checkpoint/{}_actor_checkpoint.pth'.format(i_episode-100))
        torch.save(agent.critic_local.state_dict(), 'checkpoint/{}_critic_checkpoint.pth'.format(i_episode-100))
        success_score=int(np.mean(scores_window))+1
        with open('checkpoint/{}_scores.pkl'.format(i_episode-100),'wb') as f:
            pickle.dump(scores,f)
        print(" {0} {0}\n|   Episode Number\t|   Avg Score({1})\t|\n {2} {2}".format('_'*23,MONITOR_INTERVAL,'-'*23))
    i_episode+=1

torch.save(agent.actor_local.state_dict(), 'actor_checkpoint.pth')
torch.save(agent.critic_local.state_dict(), 'critic_checkpoint.pth')
with open('scores.pkl','wb') as f:
    pickle.dump(scores,f)

### 4. Observing the trained agent.

The trained agent can now be observed.

In [12]:
from collections import deque
from Agent import Agent
from unityagents import UnityEnvironment
import numpy as np
import torch
import pickle
    

env_info = env.reset(train_mode=False)[brain_name]
state = env_info.vector_observations[0]
done = False

agent = Agent(state_size, action_size, seed=0)
#Load trained weights

#Weights were trained on the GPU. Need to convert for cpu
agent.actor_local.load_state_dict(torch.load('actor_checkpoint.pth',map_location={'cuda:0': 'cpu'}))
agent.critic_local.load_state_dict(torch.load('critic_checkpoint.pth',map_location={'cuda:0': 'cpu'}))

score = 0
while not done:
    action = agent.get_action(state,add_noise=False)[0]
    env_info = env.step(action)[brain_name]
    next_state = env_info.vector_observations[0]
    reward = env_info.rewards[0]
    done = env_info.local_done[0]
    agent.step(state,action,reward,next_state,done)
    score += reward
    state = next_state
print('You obtained a score of: {}'.format(score))

You obtained a score of: 39.439999118447304


When finished, you can close the environment.

In [ ]:
env.close()